<a href="https://colab.research.google.com/github/woongjichoi/flask/blob/master/stttts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import division

!pip install flask-ngrok
!pip install flask-cors
!pip install flask==0.12.2
from flask_cors import cross_origin
from flask import Flask, render_template, request, redirect, jsonify

!pip install google-cloud-speech


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/drive/MyDrive/august-broker-320406-301a73102397.json" # STT API 키 

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1).
libportaudiocpp0 is already the newest version (19.6.0-1).
portaudio19-dev is already the newest version (19.6.0-1).
libasound2-dev is already the newest version (1.1.3-5ubuntu0.6).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 57 not upgraded.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
sample_file="/content/drive/MyDrive/cheeseburger.wav" # STT 샘플 파일 구글 드라이브에서 가져오기 

In [ ]:
pip install google-api-core

In [ ]:
pip install pyttsx3

In [ ]:
import pyttsx3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path="/content/drive/MyDrive"

In [ ]:
!sudo apt-get update && sudo apt-get install espeak

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)

engine=pyttsx3.init() # 콜백 에러 방지하기 위함 

def text_to_speech(text, gender): # TTS
    voice_dict = {'Male': 0, 'Female': 1}
    code = voice_dict[gender]
    
    #engine=pyttsx3.init()
    engine.setProperty('rate', 150)  # 말하기 속도
    engine.setProperty('volume', 1)  # 볼륨 (min=0, MAX=1)
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[code].id)

    engine.say(text)
    engine.save_to_file(text, path+"/sample_audio.mp3")
    engine.runAndWait()

def transcribe_streaming(stream_file): # STT
    """Streams transcription of the given audio file."""
    import io
    from google.cloud import speech

    client = speech.SpeechClient()

    with io.open(stream_file, "rb") as audio_file:
        content = audio_file.read()

    # In practice, stream should be a generator yielding chunks of audio data.
    stream = [content]

    requests = (
        speech.StreamingRecognizeRequest(audio_content=chunk) for chunk in stream
    )

    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="ko-KR",
    )

    streaming_config = speech.StreamingRecognitionConfig(config=config)

    # streaming_recognize returns a generator.
    responses = client.streaming_recognize(
        config=streaming_config,
        requests=requests,
    )

    for response in responses:
        # Once the transcription has settled, the first result will contain the
        # is_final result. The other results will be for subsequent portions of
        # the audio.
        for result in response.results:
            #print("Finished: {}".format(result.is_final))
            #print("Stability: {}".format(result.stability))
            alternatives = result.alternatives
            # The alternatives are ordered from most likely to least.
            for alternative in alternatives:
              answer=alternative.transcript
                #print("Confidence: {}".format(alternative.confidence))
                #print(u"Transcript: {}".format(alternative.transcript))

    return answer

@app.route("/", methods=["GET", "POST"]) # STT
def index():
  answer=transcribe_streaming(sample_file)
  return answer

# TTS
# TTS url 접속 시 ~.ngrok.io/tts?voices=Male  
@app.route("/tts", methods=["GET"])
@cross_origin()
def homepage():
  text="치즈버거 하나 주세요"
  gender=request.args.get('voices')
  text_to_speech(text, gender)
  return path+"/sample_audio.mp3"

if __name__ == "__main__":
    app.run()
    

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9745-35-243-177-196.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [11/Nov/2021 07:18:54] "GET /tts?voices=Male HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2021 07:18:55] "GET /favicon.ico HTTP/1.1" 404 -
